In [ ]:
import awkward as ak
import numpy as np
events = ak.from_parquet("./output/example_photons.parquet")

## Let's see how many events we simulated

In [ ]:
print(f"Number of events simulated: {len(events)}")

## We can access the true Monte Carlo information by looking in the `mc_truth` field

Let's do this now for the a single event. Note that in what follows, you can replace `event` with `events` to get the information for all events

In [ ]:
event = events[7]
event["mc_truth"]

In [ ]:
# Convenience dictionary for reading the integer particle encoding
pdg_dict = {
    11: "E-",
    12: "NuE",
    13: "Mu-",
    14: "NuMu",
    15: "Tau-",
    16: "NuTau",
    -11: "E+",
    -12: "NuEBar",
    -13: "Mu+",
    -14: "NuMuBar",
    -15: "Tau+",
    -16: "NuTauBar",
    -2000001006: "Hadrons"
}
# Convenience dictionary for reading interaction type
interaction_dict = {
    0: "Neutral current",
    1: "Charged current",
    2: "Glashow resonance"
}

## The properties of the initial state have the form `initial_state_*`.

Let's take a peak at some of these for our event !

In [ ]:
f"We can look at {[x for x in event['mc_truth'].fields if 'initial' in x]}"

In [ ]:
init_type = event["mc_truth", "initial_state_type"]
init_e = event["mc_truth", "initial_state_energy"]
print(f"This initial particle was a {pdg_dict[init_type]} with energy {init_e} GeV.")

In [ ]:
init_vertex = np.array([
    event["mc_truth", "initial_state_x"],
    event["mc_truth", "initial_state_y"],
    event["mc_truth", "initial_state_z"]
])
print(f"The interaction vertex was at {init_vertex} m")

## Enough of that, let's look at the final state information

In [ ]:
f"We can look at {[x for x in event['mc_truth'].fields if 'final' in x]}"

Since we can't know *a priori* how many final states there will be---think $\tau$ decay and such---these must be stored in a list. Let's see what type of particles these were. We can also see if this matches with the expected interaction type.

In [ ]:
final_type = event["mc_truth", "final_state_type"]
print(f"The final products of this interaction are {[pdg_dict[x] for x in final_type]}")

interaction_type = event["mc_truth", "interaction"]
print(f"This was a {interaction_dict[interaction_type].lower()} interaction.")

In [ ]:
final_e = event["mc_truth", "final_state_energy"]
print(f"The final particles had energies {final_e} GeV")

## Now we'll check out the photons that arrived at the detector.

Let's see what fields are available to us.

In [ ]:
event["photons"].fields

We can find the number of photons that reached an OM by checking the length of any of the array that are stored in these fields

In [ ]:
print(f"The first event produced {len(event['photons', 't'])} photons that reached an OM")

unique_om = list(set(x for x in zip(event["photons", "string_id"], event["photons", "sensor_id"])))
print(f"The number of OMs that saw light is {len(unique_om)}")

In [ ]:
times = event["photons", "t"]
print(f"The first photon arrived at {np.min(times)} ns and the last one arrived at {np.max(times)} ns")

Finally, although it is a bit convoluted, we are able to find which of the final states produced each photon.

In [ ]:
which_final = event["mc_truth", "final_state_type", event["photons", "id_idx"]-1]
print([pdg_dict[x] for x in which_final])

## Finally, let's get the `Prometheus` configuration information out of this file

In [ ]:
import pyarrow.parquet as pq
import json

config = json.loads(pq.read_metadata('./output/example_photons.parquet').metadata[b'config_prometheus'])
for k, v in config.items():
    print(k)
    print(v)
    print()